<a href="https://colab.research.google.com/github/Colsai/DATA-601-Wan/blob/main/DATA601_WK3_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DATA601: Homework 3
- Generate 100 (n) rows of randomly-chosen people.

I'm going to assume that we are looking at *only faking US citizens*.  
One of the issues with other countries is that phone numbers, postal codes, popular names, etc. are all very regionally/country-dependent.   
Since I am randomizing from tables, it would NOT be a good idea to broaden this scope as being representative of world citizens, which would take far more complexity.

In [665]:
#Import Python Packages
import random
import pandas as pd
import numpy as np

Making sure that all phone numbers will be of len 3 + 7 = 10.  
https://stackoverflow.com/questions/733454/best-way-to-format-integer-as-string-with-leading-zeros

## 1a. Categories that we will look at
•Email addresses. Must have @  
•Phone  numbers  
•Home  Address  
•Person's  name  
•Year born. Use realistic values.  
•Number of kids. Use realistic values.   
•Categorical variable: rent or own?  
•Annual income. Optional challenge:  Use a non-uniform distribution  
•Number  of speeding  tickets in past year.Optional challenge:  Use a   non-uniform distribution  

US Area Codes are between 0-200. Since we don't really know where they got their phone number, it is possible for phone number not to be near or equal to the home address

- I'm not 100% sure that the Z-fill will work, but I think it should

## 2. Fake Phone Number
A US Phone number should be XXXXXXXXXX or 10 digits long.  
*Note: For a more-accurate phone number, it would be better to tie the 
area code to the fake random address, but it is difficult to overlap the area code to the actual place of residence, since the locations are also fake.*

In [666]:
def gen_fake_phone():
    area_code = str(random.randint(200,999))
    digits = random.randint(0,9999999)

#Add zeroes to random numbers that are less than 7 in length (Force length = 7)
    if len(str(digits)) < 7:
        extra_digits = abs(len(str(digits)) - 7)
        digits = ("0" * extra_digits) + str(digits)

    fake_phone = int(str(area_code) + str(digits))
    
    return fake_phone

### 2b. Test the function
Generate 5 random phone numbers

In [667]:
for i in range(0,5):
    print(gen_fake_phone())

3151805984
6351654095
2109842488
5849261366
8041065074


## 3a. Generate a Random Name
- Randomly chooses a gender.
- Attaches a random common name from one of the gender lists to the person
- Adds a random pick from common Last Names  

In [668]:
#Read the HTML of the most popular first names
df = pd.read_html('https://www.ksat.com/features/2020/12/31/these-were-the-50-most-popular-baby-names-in-2020/')[0]
girls_fnames = df['Girls'].to_list()
boys_fnames = df['Boys'].to_list()

#List of Last Names
df2 = pd.read_html('https://en.wikipedia.org/wiki/List_of_most_common_surnames_in_North_America')[7]
flast_name_list = df2['Name'].to_list()

In [669]:
#How many possible combinations of random names are there?
print((len(girls_fnames) + len(boys_fnames)) * len(flast_name_list))

10000


### Randomly choose a gender first, then assign a name based on random gender
- (the ratio is 51f/49m)  
Source: http://statisticstimes.com/demographics/country/us-sex-ratio.php#:~:text=Gender%20Ratio%20in%20the%20United%20States%20in%202020%20is%2097.948,to%2049.48%20percent%20male%20population.
- Were we to make this more-realistic over an amount of a million people +, we could also add n/nonbinary (and pick an inclusive list of first names).

In [670]:
#Let's assume that names should go with genders (gender would be defined first, prior to birth, then naming would come after)
def gen_fake_full_name():
    fake_gender = random.choices(('M','F'),(49,51))[0]

    if fake_gender == 'M':
      fname = random.choice(boys_fnames)

    elif fake_gender == 'F':
      fname = random.choice(girls_fnames)

    else:
        #add non-gendered names list here
        pass

    flast_name = random.choice(flast_name_list)

    fake_full_name = "".join([fname, " ",flast_name]) 
    return fake_full_name

### 3b. Test full name: 
Test the gen_fake_full_name function

In [671]:
gen_fake_full_name()

'Wyatt Clark'

## 4a. Address (Locations)
So, what are the most common places for people to live? These places should be statistically most likely for our example fake data to be from, so they could be good candidates. I'll scrape a single table from Wikipedia.

(If we wanted state abbreviations, we could also add something like: https://gist.github.com/rogerallen/1583593, which is a full dictionary of state -> abbrv)

In [672]:
#Dataframe 'Locations' table 10 is the list for the USA
df_locations = pd.read_html('https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population')[10]
df_locations.head()

,City,State,2010 Census,2000 Census,Change,2010 land area,2010 land area.1,2010 population density,2010 population density.1,ANSI,Location
0,Paradise,Nevada,223167,186070,+19.94%,46.717 sq mi,120.996 km2,"4,777/sq mi","1,844/km2",2409023.0,36°04′51″N 115°08′12″W﻿ / ﻿36.08073°N 115.1368°W
1,Arlington,Virginia,207627,189453,+9.59%,25.98 sq mi,67.288 km2,"7,992/sq mi","3,086/km2",NaN,38°52′48″N 77°10′59″W﻿ / ﻿38.880°N 77.183°W
2,Sunrise Manor,Nevada,189372,156120,+21.30%,33.352 sq mi,86.381 km2,"5,678/sq mi","2,192/km2",2410035.0,36°10′43″N 115°02′56″W﻿ / ﻿36.1785°N 115.0490°W
3,Spring Valley,Nevada,178395,117390,+51.97%,33.226 sq mi,86.055 km2,"5,369/sq mi","2,073/km2",2408797.0,36°05′55″N 115°15′43″W﻿ / ﻿36.0987°N 115.2619°W
4,Metairie,Louisiana,138481,146136,−5.24%,23.222 sq mi,60.145 km2,"5,963/sq mi","2,302/km2",2403282.0,29°59′52″N 90°10′40″W﻿ / ﻿29.9978°N 90.1779°W


In [673]:
#Let's focus in only on locations for the city and state. The other parts aren't important
df_city_state = df_locations[['City','State']]
df_city_state.head()

,City,State
0,Paradise,Nevada
1,Arlington,Virginia
2,Sunrise Manor,Nevada
3,Spring Valley,Nevada
4,Metairie,Louisiana


So that it is somewhat real...

What is the highest house number: https://able2know.org/topic/160467-1#:~:text=The%20highest%20house%20number%20I,108%20miles%20from%20Key%20West?

(33007, which I will use within choosing house numbers as the high bound).

Unlike some of the other random statements, I will use pd.sample to pull results from wikipedia's table, since city name + state are connected.

### About fake streets
I was having some issues using soup to scrape the street names, so I just added them as a list. If I were to revise this, I would probably go back in and try to scrape a list of popular street names from a website.

Sources:
https://www.nlc.org/resource/most-common-u-s-street-names/  
https://christinarebuffet.com/blog/american-street-abbreviations/


In [674]:
def gen_fake_address():
    #Randomize what they might live in... (in case we need apartment numbers)
    fake_housing_type = random.choice(["House","Apartment"])

    #Fake street number/street name/street type
    fake_number = random.randint(1,33007)
    fake_streetname = random.choice(['First',
                                     'Second',
                                     'Third',
                                     'Fourth',
                                     'Fifth',
                                     'Sixth',
                                     'Seventh',
                                     'Eighth',
                                     'Ninth',
                                     'Tenth',
                                     'Park',
                                     'Oak',
                                     'Cedar'])
    
#This is not an efficient way to do this, but it is easy to implement
    fake_street_type = random.choice(['St.', 
                                      'Ave', 
                                      'Ln.', 
                                      'Place',
                                      'Blvd.',
                                      'Dr.',
                                      ])

    #Let's sample a city using pd.sample()
    sample_city = df_city_state.sample(1)
    fake_city = sample_city['City'].to_string(index = False)
    fake_state = sample_city['State'].to_string(index = False)

    #The corresponding zip code should probably match up, too
    #fake_zip = random.randint(10000,90000)

    if fake_housing_type == 'Apartment':
        fake_apartment_number = random.randint(0,9999)
        fake_full_address = f"{fake_number} {fake_streetname} {fake_street_type}, Apt {fake_apartment_number},{fake_city},{fake_state}"

    else:
        fake_full_address = f"{fake_number} {fake_streetname} {fake_street_type},{fake_city},{fake_state}"

    return str(fake_full_address)

### 4b. Test Fake Address
Address should be number street street type city state

In [675]:
gen_fake_address()

'13725 Tenth St., Apt 6076, Spring Valley, Nevada'

Source: https://www.geeksforgeeks.org/compute-the-weighted-average-of-a-given-numpy-array/

## 5. Children
https://www.statista.com/statistics/718084/average-number-of-own-children-per-family/


Children can be seen as
Is it a family household? 
https://www.prb.org/u-s-household-composition-shifts-as-the-population-grows-older-more-young-adults-live-with-parents/#:~:text=In%201960%2C%2085%20percent%20of,35%20percent%20during%20this%20period.
(65% are)

If it is a family household, do they have children?
https://www.statista.com/statistics/242074/percentages-of-us-family-households-with-children-by-type/
(40%)

What number of children do families typically have?
https://www.pewresearch.org/fact-tank/2018/08/09/middle-children-have-become-rarer-but-a-growing-share-of-americans-now-say-three-or-more-kids-are-ideal/

In [676]:
def gen_fake_children(age):
    if age >= 15:
        #Chance of children in a family = if family household AND if family household has children
        family_household_chance = .65
        children_chance = .4

        #Chance of children
        chance_of_children = family_household_chance * children_chance

        #Checks possibility of children
        children_exist = random.choices(['Yes','No'],[chance_of_children,1-chance_of_children])[0]

        if children_exist == 'Yes':
            num_kids = random.choices([1,2,3,4],[21,41,23,15])[0]

        elif children_exist == 'No':
            num_kids = 0

        else:
            num_kids = 0

    else:
        num_kids = 0
        
    return num_kids

In [677]:
gen_fake_children(15)

0

## 6. Age (Years born in)

In [678]:
def gen_year_born():
    current_year = 2021
    age = random.randint(0,100)
    birth_year = current_year - age 
    
    return birth_year

## 7. Speeding Tickets
Background Information:
What is the average number of speeding tickets? 
- First, the average number of speeding tickets per year is about ~16% (1/6)
Source: https://modernattorney.com/blog/facts-statistics-about-speeding-tickets

- Second, if you get more than 4 speeding convictions in 3 years, you have your license revoked. So, people cannot have more than 4/3 = 1.3 * years driving in tickets (that would be the max).

## How I calculated Tickets:
Every ticket has a 16 chance.  
Number of ticket chance = ticket_chance ** number of times occurring.  
This is only up until 4 tickets. 



In [679]:
#Roughly Two Ticket Chance
1/6 * 1/6

0.027777777777777776

In [680]:
#Three Ticket Chance
1/6 * 1/6 * 1/6

0.004629629629629629

In [681]:
#Generate tickets
def gen_fake_tickets(age):
    if age >= 16:
        #Chance of a ticket
        ticket_chance = .16

        ticket_count = random.choices([0,1,2,3],[1-ticket_chance,(ticket_chance)-(ticket_chance **2 + ticket_chance **3),ticket_chance ** 2,ticket_chance ** 3])[0]

    else:
        ticket_count = 0 #Cannot get tickets because not of age to drive

    return ticket_count

### Generate Fake Tickets
It should be very rare to see ticket numbers at 2 or 3. It is rare enough to see a ticket at 1, with only 1/6.

In [682]:
for i in range(0,10):
    tickets = gen_fake_tickets(18)
    print(tickets)

0
0
0
0
0
1
0
0
0
0


#
https://www.census.gov/housing/hvs/files/currenthvspress.pdf

## 8. Rent or Own a Home
This looks at the chance of whether or not someone can own a home. The ratio is about 59-30 https://www.census.gov/housing/hvs/files/currenthvspress.pdf .

If current age is less than 18, the person should not be able to buy or own a home (since they would have to enter into a legal agreement generally requiring age of majority).

https://www.thebalance.com/when-is-the-best-age-to-buy-a-home-4163871#:~:text=At%20What%20Age%20Can%20You,legal%20age%20signs%20the%20contracts.

In [683]:
def gen_home_status(age):
    if age >= 18:
        #Categorical Variable (Rent or Own)
        housing_status = random.choices(['Rent','Own'],[58.6,30.4])[0]
    else:
        housing_status = 'N/A'

    return housing_status

## 9. Annual Income

In [684]:
def gen_fake_annual_income(age):
    if age >= 16:
        #Annual Income
        fake_income = random.randint(15000,250000)
        fake_income = str(f"${fake_income}")
    else:
        fake_income = '0'

    return fake_income

## 10. Email Addresses
There are a lot of ways we could do email addresses: as random combinations of characters, of words, etc. 
A lot of these names might be common, so some ways people make unique email accounts are:
- adding numbers to the end of their name
- shortening parts of their name for their email
- adding different joining characters for their email.

I'll try to add these three elements here:

In [685]:
def gen_fake_email(fake_name):
    domain = random.choice(["gmail.com",
                            "yahoo.com",
                            "aol.com",
                            "hotmail.com",
                            "outlook.com",
                            "protonmail.com"])

    num_suffix = random.randint(0,999)

    #Split the string back into two names
    fake_name = fake_name.split(" ")

    #Occasionally truncate email string, which people do for unique email addresses
    for i in range(0,len(fake_name)):
        fake_name[i] = fake_name[i][0:random.randint(int(len(fake_name[i])-1),len(fake_name[i]))]

    #Adding different ways to join email address
    email_separators = random.choice(["","_",".","-"])

    #Re-joiing the fake email addresses
    fake_name_joined = f'{email_separators}'.join(fake_name)

    #Creating a full email address
    email_address = (fake_name_joined + str(num_suffix) + "@" + domain).lower()
    
    return email_address

https://stackoverflow.com/questions/12453580/how-to-concatenate-items-in-a-list-to-a-single-string

## 11 Recreate as a .CSV
Each Row Will Include all of the data

In [692]:
number_people = 1000 #Change this number to the number of fake people
people_list = []

for person in range(0,number_people):
    #These variables are generated first, because they affect other variables.
    fake_full_name = gen_fake_full_name()
    year_born = gen_year_born()

    #Age variable is used for fake_children, annual_income, and home_status.
    age = 2021 - year_born

    people_list.append([fake_full_name,
                        gen_fake_address(),
                        gen_fake_phone(),
                        year_born,
                        gen_home_status(age),
                        gen_fake_tickets(age), 
                        gen_fake_children(age),
                        gen_fake_annual_income(age),
                        gen_fake_email(fake_full_name),
                        ])

### Test the list output

In [693]:
#Test the list output
for person in people_list[0:5]:
    print(person)

['Chloe Peterson', '3495 Oak Ave, Spring Valley, Nevada', 8862469286, 1978, 'Rent', 1, 3, '$186003', 'chloepeterso738@gmail.com']
['Avery Kelly', '22915 Oak Place, Apt 2235, Paradise, Nevada', 9937257128, 1941, 'Own', 1, 0, '$237595', 'aver_kell22@yahoo.com']
['Miles Hill', '15693 Ninth Blvd., Spring Valley, Nevada', 5692580299, 1958, 'Rent', 1, 3, '$177008', 'miles-hil623@outlook.com']
['Julian Myers', '6777 Ninth Blvd., East Los Angeles, California', 7927172384, 1956, 'Rent', 0, 0, '$34547', 'julia.myers205@yahoo.com']
['Emilia Roberts', '20736 Oak Blvd., Apt 8612, Paradise, Nevada', 7925064799, 2003, 'Rent', 2, 0, '$177055', 'emiliaroberts714@outlook.com']


### Convert it into a dataframe

In [694]:
#Columns and create dataframe
name_columns = ["Name", 
                "Address", 
                "Phone Numbers",
                "Year Born", 
                "Home Status",
                "Speeding Tickets", 
                "Children",
                "Annual Income",
                "Email"]

df_full = pd.DataFrame(people_list, columns = name_columns)
df_full.head(15)

,Name,Address,Phone Numbers,Year Born,Home Status,Speeding Tickets,Children,Annual Income,Email
0,Chloe Peterson,"3495 Oak Ave, Spring Valley, Nevada",8862469286,1978,Rent,1,3,$186003,chloepeterso738@gmail.com
1,Avery Kelly,"22915 Oak Place, Apt 2235, Paradise, Nevada",9937257128,1941,Own,1,0,$237595,aver_kell22@yahoo.com
2,Miles Hill,"15693 Ninth Blvd., Spring Valley, Nevada",5692580299,1958,Rent,1,3,$177008,miles-hil623@outlook.com
3,Julian Myers,"6777 Ninth Blvd., East Los Angeles, California",7927172384,1956,Rent,0,0,$34547,julia.myers205@yahoo.com
4,Emilia Roberts,"20736 Oak Blvd., Apt 8612, Paradise, Nevada",7925064799,2003,Rent,2,0,$177055,emiliaroberts714@outlook.com
5,Penelope Kelly,"20385 Third Blvd., Apt 9869, Metairie, Louisiana",8645360519,1969,Rent,1,4,$213927,penelop.kell894@gmail.com
6,Layla Thompson,"14195 Fourth Dr., Apt 48, East Los Angeles, Ca...",3142162523,2003,Rent,0,3,$36397,layl_thompson510@protonmail.com
7,Aiden Reed,"27465 Fifth St., Brandon, Florida",7703844872,2000,Rent,1,0,$72531,aide_reed212@gmail.com
8,Mason Baker,"11017 Third Ln., Apt 9005, Metairie, Louisiana",3801122443,1997,Rent,0,0,$189706,mason-bake438@protonmail.com
9,Isaac Allen,"30178 Seventh Blvd., Spring Valley, Nevada",3742588735,1999,Rent,1,0,$241604,isaa_allen574@aol.com


In [695]:
#Export as csv
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html

df_full.to_csv("fake_people_data.csv")